In [1]:
from datetime import timedelta
import json

import numpy as np
import pandas as pd
import plotly.express as px

import sys
sys.path.append("../")

from garminexport.garminclient import GarminClient


In [4]:
client = GarminClient()
client.connect()

In [8]:
# Given dates
d0 = "2024-05-01"
d1 = "2024-05-03"

# Generate a list of days
days = pd.date_range(start=d0, end=d1, freq='D')

# Convert the list of days to a format like "YYYY-MM-DD"
days = [day.strftime('%Y-%m-%d') for day in days]

DF_ = []

for day in days:
  response = client.session.get(
            f"https://connect.garmin.com/wellness-service/wellness/dailyStress/{day}")
      
  X = json.loads(response.text)["bodyBatteryValuesArray"]
  X = pd.DataFrame(X,columns=['minutes','type','body score','version'])
  X['minutes'] = np.array([0] +list(np.diff(X['minutes']/1000)))/60
  X['minutes'] =np.cumsum(X['minutes'])

  # Example start datetime
  start_datetime = pd.Timestamp(json.loads(response.text)["startTimestampLocal"])
  
  X['datetime'] = start_datetime + pd.to_timedelta(X['minutes'], unit='m')
  DF_.append(X)
  
DF = pd.concat(DF_)


px.line(x='datetime',y='body score',data_frame=DF)